In [ ]:
import pandas as pd
df = pd.read_csv('data/amazon_cellphones.csv')
df

In [ ]:
df.isna().sum()

### clean the document

In [ ]:
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

stop = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

corpus = df['reviewText'][~df['reviewText'].isna()]

corpus = corpus.str.lower().str.replace('[^\w\s]','')
corpus = corpus.str.split()
corpus = corpus.apply(lambda x: [lemmatizer.lemmatize(item) for item in x if item not in stop])
corpus = corpus.apply(lambda x: [item for item in x if not item.isnumeric()])
# corpus = [' '.join(i) for i in corpus]
corpus

### create ngrams

In [ ]:
from gensim.models.phrases import Phrases

bigram = Phrases(corpus, min_count=5, threshold=0.2)
bigrams = [bigram[item] for item in corpus]
corpus = [bigram[item] for item in bigrams]

### create embeddings with w2v

In [ ]:
from gensim.models import Word2Vec

#sg: training algorithm. 1 for skipgram, 0 for cbow
%time w2v_model = Word2Vec(sg=0, sentences=corpus, size=100, window=5, min_count=5, iter=5)

### create embeddings with fastText

In [ ]:
from gensim.models import FastText

%time ft_model = FastText(sg=1, sentences=corpus, size=100, window=5, min_count=5, iter=5, min_n=4, max_n=6)
# ft_model.build_vocab(sentences=corpus)

In [ ]:
from gensim.models import KeyedVectors 
w2v_model.wv.save_word2vec_format('models/w2v_model.bin', binary=True)
ft_model.wv.save_word2vec_format('models/ft_model.bin', binary=True)